In [1]:
import numpy as np
import os, sys
sys.path.append(os.pardir)
from load_foot import Load_data, make_data
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable
from mymodule.layers import LSTM, T_CNN
from mymodule.utils import data_loader
from tensorboardX import SummaryWriter

In [2]:
epochs = 100
seq_len = 128
batch_size = 4

os.chdir('data')
data = Load_data(train_mat='train_foot2.mat',
                 test_mat='test_foot2.mat',
                 train_label_mat='label_foot.mat',
                 test_label_mat='label_foot.mat')
train_x, train_t, test_x, test_t = data.get_data2d(seq_len)
train_diff = np.zeros(train_x.shape)
test_diff = np.zeros(test_x.shape)
train_diff[:,1:seq_len,:] = np.diff(train_x, axis=1)
test_diff[:,1:seq_len,:] = np.diff(test_x, axis=1)

### (batch, seq, 10 = 5 + 5)
train_ = np.c_[train_x, train_diff].reshape(-1, 1, 128, 10)
test_ = np.c_[test_x, test_diff].reshape(-1, 1, 128, 10)

train_ = torch.from_numpy(train_)
test_ = torch.from_numpy(test_)

train_loader = data_loader(train_, train_t, batch_size=batch_size, gpu=True)
test_loader = data_loader(test_, test_t, batch_size=batch_size)

os.chdir('..')

In [3]:
print(train_.shape)
print(test_.shape)
print(train_t.shape)
print(test_t.shape)

(14209, 1, 128, 10)
(14209, 1, 128, 10)
(14209,)
(14209,)


In [4]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
#         self.cnn = torch.nn.ModuleList([torch.nn.Conv2d(1, 32, (4, 1), bias=False)])
        
        self.cnn = torch.nn.Conv2d(1,32,2)
    
    def forward(self, x):
#         for l in enumerate(self.cnn):
#             x = l(x)
        x = self.cnn(x)
        return x

In [5]:
model = CNN()

In [7]:
y = model(Variable(train_x))
print(y.shape)

RuntimeError: Variable data has to be a tensor, but got numpy.ndarray